<a href="https://colab.research.google.com/github/Mehul-Agrawal410/Mini-Shakespeare-Chatbot/blob/main/GPTbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [32]:
!wget https://github.com/Mehul-Agrawal410/Mini-Shakespeare-Chatbot/blob/main/input.txt

--2023-07-15 10:50:43--  https://github.com/Mehul-Agrawal410/Mini-Shakespeare-Chatbot/blob/main/input.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4276 (4.2K) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   4.18K  --.-KB/s    in 0s      

2023-07-15 10:50:43 (39.6 MB/s) - ‘input.txt’ saved [4276/4276]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [5]:
ch_to_ix = {c:i for i, c in enumerate(chars)}
ix_to_ch  = {i:c for i,c in enumerate(chars)}
encode = lambda s: [ch_to_ix[i] for i in s]
decode = lambda s: ''.join([ix_to_ch[i] for i in s])

In [6]:
a = encode('hello there')
print(a)
print(''.join(decode(a)))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [7]:
import torch

data = torch.tensor(encode(text), dtype = torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9 * len(data))
train = data[:n]
val = data[n:]

In [9]:
batch_size = 8
block_size = 16
max_iters = 3000
eval_interval = 300  #just for calculating loss at fixedd intervals
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
num_embd = 64   #embedding size of every unique character in vocabulary

In [10]:
def get_batch(data):
  indexes = torch.randint(high = len(data) - block_size, size = (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in indexes])
  y = torch.stack([data[i+1:i+block_size+1] for i in indexes])
  return x, y

In [11]:
x_batch, y_batch = get_batch(train)
print(x_batch, y_batch)

tensor([[ 1, 39,  1, 61, 53, 56, 42,  1, 61, 47, 58, 46,  1, 53, 52, 43],
        [53, 59, 56,  5, 42,  1, 44, 53, 56,  1, 46, 47, 57,  1, 40, 59],
        [46, 47, 51, 10,  0, 28, 56, 39, 63,  1, 19, 53, 42,  1, 61, 43],
        [43, 40, 63,  6,  0, 14, 43, 41, 39, 59, 57, 43,  1, 21,  1, 61],
        [47, 51, 43,  7, 54, 50, 43, 39, 57, 43, 56, 57,  6,  1, 44, 50],
        [39, 56,  1, 51, 43,  6,  1, 52, 53, 40, 50, 43,  1, 50, 53, 56],
        [61, 43,  1, 61, 47, 50, 50,  1, 54, 56, 53, 41, 50, 39, 47, 51],
        [47, 52, 43,  1, 53, 61, 52,  1, 45, 53, 53, 42,  1, 61, 47, 50]]) tensor([[39,  1, 61, 53, 56, 42,  1, 61, 47, 58, 46,  1, 53, 52, 43,  1],
        [59, 56,  5, 42,  1, 44, 53, 56,  1, 46, 47, 57,  1, 40, 59, 56],
        [47, 51, 10,  0, 28, 56, 39, 63,  1, 19, 53, 42,  1, 61, 43,  1],
        [40, 63,  6,  0, 14, 43, 41, 39, 59, 57, 43,  1, 21,  1, 61, 47],
        [51, 43,  7, 54, 50, 43, 39, 57, 43, 56, 57,  6,  1, 44, 50, 39],
        [56,  1, 51, 43,  6,  1, 52, 

In [12]:
class BigramLangModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
    self.langModel_head = nn.Linear(num_embd, vocab_size)

  def forward(self, index, targets = None):

    batch, seq_len = index.shape
    #index and target are (batch_size, seq_len) tensors
    tok_embd = self.token_embedding_table(index)  #(batch_size, seq_len, emb_size)
    logits = self.langModel_head(tok_embd)  #(batch_size, seq_len, vocab_size)

    if targets is None:
      loss = None
    else:
      batch_size, seq_len, emb_size = logits.shape
      logits = logits.view(batch_size * seq_len, emb_size)
      targets = targets.view(batch_size * seq_len)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  #extends the time dimension by max tokens
  def generate(self, context, max_new_tokens):
    #context is (batch, time) array of indices in current context
    for _ in range(max_new_tokens):
      logits, loss = self(context)
      logits = logits[:,-1,:] # pluck out the last element in "time" as that will predict next elem
      probs = F.softmax(logits, dim=1)
      index_next = torch.multinomial(probs, num_samples = 1) #Returns num_samples rows indices sampled from the multinomial probability "probs"
      context = torch.cat([context, index_next], dim=1) #(batch, time+1)
    return context


In [13]:
model = BigramLangModel()
raw_outs, loss = model(x_batch, y_batch)
print(raw_outs.shape)
print(loss)

context = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist())) #0 cause it return (1,1) tensor

torch.Size([128, 65])
tensor(4.2876, grad_fn=<NllLossBackward0>)

jdmAj?hanBevKE3DL;$-PDxZAhERxx BL'Spc:SypDuDChnWz.u;TJzRaS:a-fV$c'pkKn'E;e3IM&,:kVzAatkii,xzaFTScH$M


In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [15]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in [train, val]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [16]:
for iter in range(max_iters):

  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses[train]:.4f}, val loss {losses[val]:.4f}")

  # sample a batch of data
  x_batch, y_batch = get_batch(train)

  # evaluate the loss
  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.3577, val loss 4.3675
step 300: train loss 2.7507, val loss 2.7505
step 600: train loss 2.5965, val loss 2.6071
step 900: train loss 2.5299, val loss 2.5649
step 1200: train loss 2.5182, val loss 2.5466
step 1500: train loss 2.5123, val loss 2.5347
step 1800: train loss 2.5063, val loss 2.5296
step 2100: train loss 2.4848, val loss 2.5170
step 2400: train loss 2.4838, val loss 2.5288
step 2700: train loss 2.4983, val loss 2.5174


In [17]:
print(decode(model.generate(torch.zeros((1,1), dtype = torch.long), 2000)[0].tolist()))


Thn mellif an o l a mouny ickery r aasinCKEd y t ingnnd e;
Thepe tthy, tongerot ths go VI sskipr igo: t'd t str tantwisarilot bath ce alamary ilghes, RI gou o athif t cisuriemathtet mm, outy, y, heay thatares y acem

t ourout th shof crer llim?
FCllave-pas:
Ant'vet and
Forear, Haioures aspe,
Tisor f D:
ND is helel h T:
TENICiofathaindeage d ar, ty hopy, da atayord hesuthike; ororale, f lu wonthemome I mese wo is sod.
Gor s s ay h ickad, t ad be thehe, atisoult!bo thasullon'ds'ssinthar:
Nong; nd by, we y y nd h
3-r s;
Me my? wean?
Habr herendot,
ANTENDsapow by Ache'T:
Thidy thar bran incrothink thas, blersuce cas.

F
I:
Aner on, gan bor;
W t ch s yiffotrt EO:
Looustha meanof inds t: we chWhathar o s s
GLyothar .

fot, be IOLuseristhinof me.
INGy o croliningad p
We thrt d s mp hin'ta miclspougoune ds, whendaurertrary h, ss tor y velou tet jcke tome thugrr g'eande, brt, lnowity EN e g!
TELThth;
Imy, ikea co t tor h be
Anghinds owh m tunathat ss; y; ond:
Thenire lesse we f wht l vene me s

In [18]:
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300  #just for calculating loss at fixedd intervals
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
num_embd = 32   #embedding size of every unique character in vocabulary
num_head = 4
num_layer = 4  #attention hyperparameter
dropout = 0.2

In [19]:
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(num_embd, head_size, bias=False)
    self.query = nn.Linear(num_embd, head_size, bias=False)
    self.value = nn.Linear(num_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # input of size (batch, time-step, channels)
    # output of size (batch, time-step, head size)
    batch_size, seq_len, emb_size = x.shape
    k = self.key(x)                                       # (batch_size, seq_len, head_size)
    q = self.query(x)                                     # (batch_size, seq_len, head_size)
    wei =  q @ k.transpose(-2, -1) * k.shape[-1]**-0.5    # (batch_size, seq_len, head_size) @ (batch_size, head_size, seq_len) ---> (batch_size, seq_len, seq_len)
                                                          # this outputs the raw affinity between all the nodes
    tril = torch.tril(torch.ones(seq_len, seq_len))
    wei = wei.masked_fill(self.tril == 0, float('-inf'))       #because we don't want the affinity with nodes ahead of current node
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out

In [20]:
class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size * num_heads, num_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

In [21]:
class FeedFoward(nn.Module):

  def __init__(self, num_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(num_embd, 4 * num_embd),
      nn.ReLU(),
      nn.Linear(4 * num_embd, num_embd),
      nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

In [22]:
class Block(nn.Module):

  def __init__(self, num_embd, num_head):
    # num_embd: embedding dimension, num_head: the number of heads we'd like
    super().__init__()
    head_size = num_embd // num_head
    self.self_attn = MultiHeadAttention(num_head, head_size)
    self.ffwd = FeedFoward(num_embd)
    self.layer_norm_1 = nn.LayerNorm(num_embd)
    self.layer_norm_2 = nn.LayerNorm(num_embd)

  def forward(self, x):
    x = x + self.self_attn(self.layer_norm_1(x))
    x = x + self.ffwd(self.layer_norm_2(x))
    return x

In [23]:
class GPTLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
    self.position_embedding_table = nn.Embedding(block_size, num_embd)
    self.blocks = nn.Sequential(*[Block(num_embd, num_head=num_head) for _ in range(num_layer)])
    self.ln_f = nn.LayerNorm(num_embd) # final layer norm
    self.lm_head = nn.Linear(num_embd, vocab_size)

    # better init, not covered in the original GPT video, but important, will cover in followup video
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
          torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

  def forward(self, idx, targets=None):
    batch_size, seq_len = idx.shape

    # idx and targets are both (batch_size,seq_len) tensor of integers
    tok_emb = self.token_embedding_table(idx) # (batch_size,seq_len,token_embd_size)
    pos_emb = self.position_embedding_table(torch.arange(seq_len, device=device)) # (seq_len, pos_embd_size)
    x = tok_emb + pos_emb # (batch_size,seq_len,C)
    x = self.blocks(x) # (batch_size,seq_len,C)
    x = self.ln_f(x) # (batch_size,seq_len,C)
    logits = self.lm_head(x) # (batch_size,seq_len,vocab_size)

    if targets is None:
      loss = None
    else:
      batch_size, seq_len, embd_size = logits.shape
      logits = logits.view(batch_size*seq_len, embd_size)
      targets = targets.view(batch_size*seq_len)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, context, max_new_tokens):
    # context is (batch_size, seq_len) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      context_cropped = context[:, -block_size:]
      # get the predictions
      logits, loss = self(context_cropped)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (batch_size, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (batch_size, C)
      # sample from the distribution
      context_next = torch.multinomial(probs, num_samples=1) # (batch_size, 1)
      # append sampled index to the running sequence
      context = torch.cat((context, context_next), dim=1) # (batch_size, seq_len+1)
    return context

In [24]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()), 'M parameters')

54977 M parameters


In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [26]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in [train, val]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [27]:
for iter in range(max_iters):

  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses[train]:.4f}, val loss {losses[val]:.4f}")

  # sample a batch of data
  x_batch, y_batch = get_batch(train)

  # evaluate the loss
  logits, loss = model(x_batch, y_batch)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step 0: train loss 4.1738, val loss 4.1739
step 300: train loss 2.5078, val loss 2.5100
step 600: train loss 2.3894, val loss 2.3949
step 900: train loss 2.3319, val loss 2.3225
step 1200: train loss 2.2694, val loss 2.2806
step 1500: train loss 2.2338, val loss 2.2496
step 1800: train loss 2.1824, val loss 2.2049
step 2100: train loss 2.1620, val loss 2.1765
step 2400: train loss 2.1486, val loss 2.1685
step 2700: train loss 2.1343, val loss 2.1585
step 2999: train loss 2.1142, val loss 2.1504


In [28]:
context = torch.zeros((1, 8), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))









HEBETUME: Is gor it ad, cAg war rould creaie, your ther'se a the etl thond you, to the mstowns, trakt
Tho, we to do sieck.


DEOLUS:
Pry, lealle fert,
Hiswith sbonboone difests.
BRIAR:
As mres; then he wall belorgue fatter wil thablan thy wo lorce,
On wo we buth
As grovem lol fridon ; With! sto It:

The thast befZanes cee meenot, the hink what are coman hall
To kn to akeplend lark prathever ay?

RAPVNRAREOKA:
Booone
A Of and.
I foris eor my liey, for nou hovo wheld
it of may, my my tan ut thy dee tho is are wilxfet I law; the thar?

And with shan grest'somy oun mined mak my quue stk blaing, thy brothere et coy, as cre ce:
sthis I qunessbe vithill.
RAVERICICETUCHARYNIV:
Nouge
Porure hourn and why,
So on sacuteen'drs of wall.
VARIBETERS:
A tha ach love thour daine blatif gook! wre mith graie, thy Gostine.

CORIVEN
Chat pous thou a tha Lot:
Harth on'gmat thur Romfers
ford pose Soustirs in of that, the ase stive gesing,
Cie?


Sith,
VINGIERS:
Lod.

GLOscitk:
Tho till sive dind bre 

In [30]:
open('outputs.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

10008